In [465]:
import yfinance as yf
import pandas as pd
import os
import datetime

BASE_DIR="/Users/dannyyu/Desktop/AI_Trader/data"
TICKERS = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'NVDA', 'TSLA', 'INTC', 'AMD', 'IBM']
START_DATE = "2022-01-01"
END_DATE = datetime.datetime.today()


ticker_obj = yf.Ticker('NVDA')
info = ticker_obj.info

recent_eps = info.get("trailingEps", None)
recent_pe = info.get("trailingPE", None)


income_stmt = ticker_obj.quarterly_financials  # Income statement
balance_sheet = ticker_obj.quarterly_balance_sheet
cashflow_stmt = ticker_obj.quarterly_cashflow

#print(income_stmt.index)
#print(balance_sheet.index)
#print(cashflow_stmt.index)

In [113]:
#income_stmt.loc["Diluted EPS", pd.Timestamp("2022-09-30")]
common_dates = income_stmt.columns.intersection(balance_sheet.columns).intersection(cashflow_stmt.columns)
print(balance_sheet.columns)

DatetimeIndex(['2025-03-31', '2024-12-31', '2024-09-30', '2024-06-30',
               '2024-03-31', '2023-12-31', '2023-09-30'],
              dtype='datetime64[ns]', freq=None)


In [5]:
common_dates = income_stmt.columns.intersection(balance_sheet.columns).intersection(cashflow_stmt.columns)
common_dates

DatetimeIndex(['2024-09-30', '2023-09-30', '2022-09-30', '2021-09-30',
               '2020-09-30'],
              dtype='datetime64[ns]', freq=None)

In [32]:
ticker=yf.Ticker("AAPL")
info = ticker.info
print(info.get("trailingPE", None))
end_date="2024-10-01"
start_date="2024-09-27"
eps=info.get("trailingEps", None)
print(eps)
hist_price = ticker.history(start=start_date, end=end_date)
print(hist_price)
hist_price=hist_price['Close'].iloc[0]
print(hist_price)

31.325039
6.43
                                 Open        High         Low       Close  \
Date                                                                        
2024-09-27 00:00:00-04:00  227.659747  228.716032  226.503807  226.992081   
2024-09-30 00:00:00-04:00  229.234200  232.183838  228.845566  232.183838   

                             Volume  Dividends  Stock Splits  
Date                                                          
2024-09-27 00:00:00-04:00  34026000        0.0           0.0  
2024-09-30 00:00:00-04:00  54541900        0.0           0.0  
226.99208068847656


In [49]:
ticker_obj=yf.Ticker("NVDA")

hist=ticker_obj.history(period="1d")#start=start_date, end=end_date)
#last_row=hist.iloc[-1]
price=hist["Close"]
price

Date
2025-05-23 00:00:00-04:00    131.289993
Name: Close, dtype: float64

In [67]:
income_stmt = ticker_obj.financials

trailing_eps=info.get("trailingPE", None)
trailing_eps

44.21026

In [69]:
trailing_pe=price/trailing_eps
trailing_pe

Date
2025-05-23 00:00:00-04:00    2.969672
Name: Close, dtype: float64

In [71]:
income_stmt.columns

DatetimeIndex(['2025-01-31', '2024-01-31', '2023-01-31', '2022-01-31'], dtype='datetime64[ns]', freq=None)

AttributeError: 'Ticker' object has no attribute 'valuation_measures'

In [168]:
def get_fundamentals_history(ticker, years=3):
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.info

    # Get quarterly and annual statements
    q_income_stmt = ticker_obj.quarterly_financials
    q_balance_sheet = ticker_obj.quarterly_balance_sheet
    q_cashflow_stmt = ticker_obj.quarterly_cashflow

    y_income_stmt = ticker_obj.financials
    y_balance_sheet = ticker_obj.balance_sheet
    y_cashflow_stmt = ticker_obj.cashflow

    # Initialize
    fundamentals = []

    # Helper to extract data row by row from financials
    def extract_fundamentals(date, is_quarterly=True):
        try:
            stmt_src = "quarterly" if is_quarterly else "yearly"
            inc = q_income_stmt if is_quarterly else y_income_stmt
            bal = q_balance_sheet if is_quarterly else y_balance_sheet
            cf  = q_cashflow_stmt if is_quarterly else y_cashflow_stmt

            revenue = inc.loc["Total Revenue", date]
            gross_profit = inc.loc["Gross Profit", date]
            ebitda = inc.loc["EBITDA", date]
            net_income = inc.loc["Net Income", date]

            operating_cashflow = cf.loc["Operating Cash Flow", date]
            total_equity = bal.loc["Common Stock Equity", date]
            total_debt = bal.loc["Total Debt", date]

            roe = net_income / total_equity if total_equity else None
            debt_to_equity = (total_debt / total_equity)*100 if total_equity else None
            operating_margin = net_income / revenue if revenue else None

            diluted_eps = inc.loc["Diluted EPS", date]
            hist_price = get_closest_price(ticker_obj, date)
            pe_ratio = (hist_price / diluted_eps) if hist_price and diluted_eps else info.get("trailingPE", None)

            return {
                "Date": date.strftime('%Y-%m-%d'),
                "DilutedEPS": diluted_eps,
                "PE": pe_ratio,
                "Revenue": revenue,
                "CashFlow": operating_cashflow,
                "EBITDA": ebitda,
                "GrossProfit": gross_profit,
                "OperatingMargin": round(operating_margin, 8),
                "ROE": round(roe, 3),
                "DebtToEquity": round(debt_to_equity, 5),
                "Source": stmt_src
            }

        except Exception as e:
            print(f"Error processing {ticker} for {date.date()} [{stmt_src}]: {e}")
            return None

    # Include most recent static values
    fundamentals.append({
        "Date": pd.Timestamp.today().strftime('%Y-%m-%d'),
        "DilutedEPS": info.get("trailingEps", None),
        "PE": info.get("trailingPE", None),
        "Revenue": info.get("totalRevenue", None),
        "CashFlow": info.get("operatingCashflow", None),
        "EBITDA": info.get("ebitda", None),
        "GrossProfit": info.get("grossProfits", None),
        "OperatingMargin": info.get("operatingMargins", None),
        "ROE": info.get("returnOnEquity", None),
        "DebtToEquity": info.get("debtToEquity", None),
        "Source": "info"
    })

    # Add quarterly reports (within last ~15 months)
    q_dates = q_income_stmt.columns.intersection(q_balance_sheet.columns).intersection(q_cashflow_stmt.columns)
    for q_date in q_dates:
        if (pd.Timestamp.today() - q_date).days <= 450:
            row = extract_fundamentals(q_date, is_quarterly=True)
            if row:
                fundamentals.append(row)

    # Add yearly reports, excluding any years already covered
    y_dates = y_income_stmt.columns.intersection(y_balance_sheet.columns).intersection(y_cashflow_stmt.columns)
    included_years = {pd.to_datetime(f["Date"]).year for f in fundamentals}

    for y_date in y_dates:
        if y_date.year in included_years or pd.Timestamp.today().year - y_date.year > years:
            continue
        row = extract_fundamentals(y_date, is_quarterly=False)
        if row:
            fundamentals.append(row)

    return pd.DataFrame(fundamentals)

In [417]:
df=get_fundamentals_history("NVDA")

In [481]:

df.index

Index(['2025-06-06', '2025-01-31', '2024-10-31', '2024-07-31', '2024-04-30',
       '2023-01-31', '2022-01-31'],
      dtype='object', name='Date')

In [126]:
"""for ticker in TICKERS:
    df=get_fundamentals_history(ticker) 
    print(ticker)
    print(df)"""

'for ticker in TICKERS:\n    df=get_fundamentals_history(ticker) \n    print(ticker)\n    print(df)'

In [523]:
def merge_table(price, fund):
    price = price.copy()
    price.index = pd.to_datetime(price.index, utc=True)
    fund = fund.copy()
    fund.index = pd.to_datetime(fund.index, utc=True)

         # Mark report source date
    fund["FundamentalReportDate"] = fund.index
    
    # Reset both so that Date is a column

    price = price.reset_index()
    fund = fund.reset_index()
    
    # Then proceed as usual
    #merged = pd.merge(price, fund, on="Date", how="left")

    merged = pd.concat([price, fund], ignore_index=True, sort=False)

    merged = merged.sort_values("Date", ascending=False)

    # Get list of fundamental columns (excluding 'Date' and 'FundamentalReportDate')
    fund_cols = fund.columns.drop(["Date", "FundamentalReportDate"], errors='ignore')
    print(fund_cols)
    # Backward fill fundamental data
    merged[fund_cols] = merged[fund_cols].bfill()
    merged["FundamentalReportDate"] = merged["FundamentalReportDate"].bfill()

    merged[fund_cols] = merged[fund_cols].ffill()
    merged["FundamentalReportDate"] = merged["FundamentalReportDate"].ffill()

    # Sort back to chronological
    merged = merged.sort_values("Date")

    # Add Year column
    merged["Year"] = merged["Date"].dt.year
    merged = merged.dropna(subset=["Close"])

    # Set Date as index
    merged.set_index("Date", inplace=True)

    return merged





def read_CSV_File(ticker,folderName):
    dir = BASE_DIR+"/"+folderName+"/"+ticker+"_"+folderName+".csv"
    table= pd.read_csv(dir,index_col=0,low_memory=False)
    return table

In [467]:
hist= ticker_obj.history(start=START_DATE, end=END_DATE,interval="1d")
hist.reset_index(inplace=True)
hist=hist.set_index("Date")
hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-01-03 00:00:00-05:00,29.762757,30.657186,29.732808,30.068220,391547000,0.0,0.0
2022-01-04 00:00:00-05:00,30.223948,30.414613,28.299327,29.238678,527154000,0.0,0.0
2022-01-05 00:00:00-05:00,28.898274,29.364457,27.484757,27.555632,498064000,0.0,0.0
2022-01-06 00:00:00-05:00,27.591572,28.388174,27.017581,28.128630,454186000,0.0,0.0
2022-01-07 00:00:00-05:00,28.091686,28.372193,27.009584,27.199251,409939000,0.0,0.0
...,...,...,...,...,...,...,...
2025-05-30 00:00:00-04:00,138.720001,139.619995,132.919998,135.130005,333170900,0.0,0.0
2025-06-02 00:00:00-04:00,135.490005,138.119995,135.399994,137.380005,197663100,0.0,0.0
2025-06-03 00:00:00-04:00,138.779999,142.000000,137.949997,141.220001,225578800,0.0,0.0


In [521]:
#hist=read_CSV_File("NVDA", "prices")
#hist
merged=merge_table(hist,df)
merged.to_csv(os.path.join(BASE_DIR, "test.csv"), index=True)
merged

Index(['DilutedEPS', 'PE', 'Revenue', 'CashFlow', 'EBITDA', 'GrossProfit',
       'OperatingMargin', 'ROE', 'DebtToEquity', 'Source'],
      dtype='object')


,Open,High,Low,Close,Volume,Dividends,Stock Splits,DilutedEPS,PE,Revenue,CashFlow,EBITDA,GrossProfit,OperatingMargin,ROE,DebtToEquity,Source,FundamentalReportDate,Year
Date,,,,,,,,,,,,,,,,,,,
2022-01-03 05:00:00+00:00,29.762757,30.657186,29.732808,30.068220,391547000.0,0.0,0.0,0.385,63.488561,2.691400e+10,9.108000e+09,1.135100e+10,1.747500e+10,0.362339,0.36600,44.45739,yearly,2022-01-31 00:00:00+00:00,2022
2022-01-04 05:00:00+00:00,30.223948,30.414613,28.299327,29.238678,527154000.0,0.0,0.0,0.385,63.488561,2.691400e+10,9.108000e+09,1.135100e+10,1.747500e+10,0.362339,0.36600,44.45739,yearly,2022-01-31 00:00:00+00:00,2022
2022-01-05 05:00:00+00:00,28.898274,29.364457,27.484757,27.555632,498064000.0,0.0,0.0,0.385,63.488561,2.691400e+10,9.108000e+09,1.135100e+10,1.747500e+10,0.362339,0.36600,44.45739,yearly,2022-01-31 00:00:00+00:00,2022
2022-01-06 05:00:00+00:00,27.591572,28.388174,27.017581,28.128630,454186000.0,0.0,0.0,0.385,63.488561,2.691400e+10,9.108000e+09,1.135100e+10,1.747500e+10,0.362339,0.36600,44.45739,yearly,2022-01-31 00:00:00+00:00,2022
2022-01-07 05:00:00+00:00,28.091686,28.372193,27.009584,27.199251,409939000.0,0.0,0.0,0.385,63.488561,2.691400e+10,9.108000e+09,1.135100e+10,1.747500e+10,0.362339,0.36600,44.45739,yearly,2022-01-31 00:00:00+00:00,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-02 04:00:00+00:00,135.490005,138.119995,135.399994,137.380005,197663100.0,0.0,0.0,0.890,134.897708,3.933100e+10,1.662900e+10,2.582100e+10,2.872300e+10,0.561669,0.27800,12.94641,quarterly,2025-01-31 00:00:00+00:00,2025
2025-06-03 04:00:00+00:00,138.779999,142.000000,137.949997,141.220001,225578800.0,0.0,0.0,0.890,134.897708,3.933100e+10,1.662900e+10,2.582100e+10,2.872300e+10,0.561669,0.27800,12.94641,quarterly,2025-01-31 00:00:00+00:00,2025
2025-06-04 04:00:00+00:00,142.190002,142.389999,139.539993,141.919998,167120800.0,0.0,0.0,0.890,134.897708,3.933100e+10,1.662900e+10,2.582100e+10,2.872300e+10,0.561669,0.27800,12.94641,quarterly,2025-01-31 00:00:00+00:00,2025


In [397]:
hist.index = pd.to_datetime(hist.index, utc=True)

hist = hist.reset_index()

ValueError: cannot insert Date, already exists